In [ ]:
import numpy as np
import sympy as sp
from sympy import init_printing
from sympy import Matrix
from sympy.interactive import printing
printing.init_printing(use_latex=True)

In [ ]:
cos=np.cos
sin=np.sin
pi=np.pi
bs="\n"
m=np.array
rd = np.rad2deg
inv=np.linalg.inv

In [ ]:
#Funciones de rotación canonicas
def rotx(ang):
    Rx=np.array([[1,0,0],
                  [0,cos(ang),-sin(ang)],
                  [0, sin(ang), cos(ang)]])
    return Rx
def roty(ang):
    Ry= np.array([[cos(ang),0,sin(ang)],
                  [0,1,0],
                  [-sin(ang), 0, cos(ang)]])
    return Ry
def rotz(ang):
    Rz= np.array([[cos(ang),-sin(ang),0],
                  [sin(ang), cos(ang), 0],
                  [0,0,1]])
    return Rz


#Funciones de rotación canonicas con valores simbolicos
def srotx(ang):
    Rx=sp.Matrix([[1,0,0],
                  [0,sp.cos(ang),-sp.sin(ang)],
                  [0, sp.sin(ang), sp.cos(ang)]])
    return Rx
def sroty(ang):
    Ry= sp.Matrix([[sp.cos(ang),0,sp.sin(ang)],
                  [0,1,0],
                  [-sp.sin(ang), 0, sp.cos(ang)]])
    return Ry
def srotz(ang):
    Rz= sp.Matrix([[sp.cos(ang),-sp.sin(ang),0],
                  [sp.sin(ang), sp.cos(ang), 0],
                  [0,0,1]])
    return Rz

#Redondear valores
def printr(x,a=None,r=None, d=None):
    if a!= None:
        Value=print(np.round(x,a))
        return Value
    elif d!=None:
        Value = print(np.round(np.rad2deg(x)))
        return Value
    elif r!=None:
        Value = print(np.round(np.deg2rad(x),4))
        return Value
    else:
        return print(np.round(x))


#Comprobar si una matriz es de rotacion

def comprobar_so3(A,P1):
    if P1==1:
        print(np.round(A.T.dot(A),2))
    elif P1==2:
        print(np.round(np.linalg.det(A),2))
    elif P1==3:
        print(np.round(A[:3,:3].T.dot(A[:3,:3]),2))
    elif P1==4:
        print(np.round(np.linalg.det(A[:3,:3]),2))
    else:
         print("Ingrese un valor valido")

def sqrt(s):
    s=np.sqrt(s)
    return s
def dot(a,b):
    print(a.dot(b))
    
def ejeang(R):
    c=(R[0,0]+R[1,1]+R[2,2]-1)/2
    s=sqrt((R[1,0]-R[0,1])**2+(R[2,0]-R[0,2])**2+(R[2,1]-R[1,2]**2))/2
    th=np.arctan2(s,c)
    u=1/(2.*sin(th))*np.array([[R[2,1]-R[1,2]],
                               [R[0,2]-R[2,0]],
                               [R[1,0]-R[0,1]]])
    return th,u

def skew(u):
    su = np.array([[0, -u[2], u[1]],
                   [u[2], 0, -u[0]],
                   [-u[1], u[0], 0]])
    return su

#Formula de rodriguez 
def rodr(th, u):
    s=skew(u)
    R=np.eye(3) + s*sin(th)+s.dot(s)*(1-cos(th))
    return R

#T given t,R
def T_tR(t,R):
    T = np.vstack((np.hstack((R,t)),np.array([[0, 0, 0, 1]])))
    return T

#R from T
def R_T(T):
    R = T[:3,:3]
    return R

def del_minz(R):
    for i in range(R.shape[0]):  
        for j in range(R.shape[1]):
            if np.round(R[i,j],3) == -0.000:
                R[i,j] = 0
    return R

#Unitary vector from vector
def unit_v(v):
    return v / np.linalg.norm(v)

#Euler angles from rotation matrix R
def R_eulerangzyz(R,s):
    
    R = del_minz(R)

    if s == '+':
        fi2 = np.arctan2(np.sqrt(R[0,2]**2+R[1,2]**2),R[2,2])
    elif s == '-':
        fi2 = np.arctan2(-np.sqrt(R[0,2]**2+R[1,2]**2),R[2,2])

    if sin(fi2) != 0:
        fi1 = np.arctan2(R[1,2]/sin(fi2),R[0,2]/sin(fi2))
        fi3 = np.arctan2(R[2,1]/sin(fi2),-R[2,0]/sin(fi2))
        return fi1, fi2, fi3
    else:
        print("Singularity")
        if (fi2/pi)%2 == 0:
            f13 = np.arctan2(R[1,0],R[0,0])
            return fi2, f13, 'fi1+fi3'
        else: 
            f31 = np.arctan2(R[0,1],R[1,1])
            return fi2, f31, 'fi3-fi1'
        
        
#Roll, pitch, yaw angles from rotation matrix R
def R_rpy(R,s):
    
    R = del_minz(R)

    if R[2,0] == 0:
        if s == '+':
            fi_p = np.arctan2(R[2,0],np.sqrt(R[2,1]**2+R[2,2]**2))
        elif s == '-':
            fi_p = np.arctan2(R[2,0],-np.sqrt(R[2,1]**2+R[2,2]**2))
    else: 
        if s == '+':
            fi_p = np.arctan2(-R[2,0],np.sqrt(R[2,1]**2+R[2,2]**2))
        elif s == '-':
            fi_p = np.arctan2(-R[2,0],-np.sqrt(R[2,1]**2+R[2,2]**2))

    if cos(fi_p) != 0:
        fi_r = np.arctan2(R[1,0]/cos(fi_p),R[0,0]/cos(fi_p))
        fi_y = np.arctan2(R[2,1]/cos(fi_p),R[2,2]/cos(fi_p))
        return fi_r, fi_p, fi_y
    else:
        print("Singularity")
        if fi_p == pi/2:
            fry = np.arctan2(R[1,2],R[0,2])
            return fi_p, fry, 'fi_r-fi_y'
        elif fi_p == -pi/2: 
            f31 = np.arctan2(-R[0,1],R[1,1])
            return fi_p, fyr, 'fi_r+fi_y'
        
        
#Axis/Angle
#Rotation matrix from U, th
def Rf_uth(u,th,au='r'):
    if au == 'd':
        th = np.deg2rad(th)

    u = unit_v(u)

    u_a = np.array([[0, -u[0,2], u[0,1]],
                    [u[0,2], 0, -u[0,0]],
                    [-u[0,1], u[0,0], 0]])

    R = np.eye(3) + u_a*sin(th) + np.dot(u_a,u_a)*(1-cos(th))
    R = del_minz(R)
    return R

#U, th from rotation matrix
def u_th_R(R,s):
    
    R = del_minz(R)

    if s == '+':
        th = np.arctan2(np.sqrt((R[1,0]-R[0,1])**2+(R[2,0]-R[0,2])**2+(R[2,1]-R[1,2])**2)/2,(np.trace(R)-1)/2)
    elif s == '-':
        th = np.arctan2(-np.sqrt((R[1,0]-R[0,1])**2+(R[2,0]-R[0,2])**2+(R[2,1]-R[1,2])**2)/2,(np.trace(R)-1)/2)
    
    if th == 0 or th == -pi or th == pi:
        print("Singularity")
    else:
        u = (1/(2*sin(th)))*np.array([[R[2,1]-R[1,2],R[0,2]-R[2,0],R[1,0]-R[0,1]]])
        return u,th 
    
    
    
#Unit Cuaternions
class  cuaternion:
    def __init__(self, w=0, ex=0, ey=0, ez=0):
        self.w = w
        self.ex = ex
        self.ey = ey
        self.ez = ez
        self.e_v = np.array([[self.ex],[self.ey],[self.ez]])

    def del_minz_c(self):
    
        if np.round(self.w,3) == -0.000:
            self.w = 0
        if np.round(self.ex,3) == -0.000:
            self.ex = 0
        if np.round(self.ey,3) == -0.000:
            self.ey = 0
        if np.round(self.ez,3) == -0.000:
            self.ez = 0
        return cuaternion(self.w, self.ex, self.ey, self.ez)
    
    def __str__(self):
        return del_minz(np.array([[self.w, self.ex, self.ey, self.ez]])).__str__()

    def __mul__(self,other):    
        cmul_w = float(self.w*other.w - np.dot(self.e_v.T,other.e_v))
        print(cmul_w)
        cmul_e = self.w*other.e_v + other.w*self.e_v + np.cross(self.e_v.T,other.e_v.T).T
        print(cmul_e)
        cmul_ex = cmul_e[0,0]
        cmul_ey = cmul_e[1,0]
        cmul_ez = cmul_e[2,0]      
        return cuaternion(cmul_w, cmul_ex, cmul_ey, cmul_ez).del_minz_c()

    def inv(self):
        return cuaternion(self.w,-self.ex,-self.ey,-self.ez).del_minz_c()

    def eye(self):
        return cuaternion(1,0,0,0)

    def to_nparray(self):
        return np.array([[self.w, self.ex, self.ey, self.ez]])

    
#Cuaternion from u,th
def cuat_uth(u,th,au='r'):
    
    if au == 'd':
        th = np.deg2rad(th)

    u = unit_v(u)

    w = cos(th/2)
    ex = u[0,0]*sin(th/2)
    ey = u[0,1]*sin(th/2)
    ez = u[0,2]*sin(th/2) 

    return cuaternion(w,ex,ey,ez).del_minz_c()


#Cuaternion from angles of x,y,z
def cuat_rotxyz(th,c,au='r'):

    if au == 'd':
        th = np.deg2rad(th)

    w = cos(th/2)

    if c == 'x':
       ex = sin(th/2)
       ey = 0
       ez = 0
    elif c == 'y':
       ex = 0
       ey = sin(th/2)
       ez = 0
    elif c == 'z':
       ex = 0
       ey = 0
       ez = sin(th/2)

    return cuaternion(w,ex,ey,ez).del_minz_c()


#Axis/angle from cuaternion
def u_th_C(C):
    
    th = 2*np.arctan2(np.linalg.norm(C.e_v),C.w)
    u = (C.e_v/np.linalg.norm(C.e_v)).T
    
    if (np.round(np.linalg.norm(C.e_v),3) == np.round(sin(th/2),3) and np.round(sin(th/2),3) == 0):
        print("Rotation angle = 0")
    else: 
        return u, th
    
    
#Rotation from cuaternion
def Rf_C(C):

    [u,th] = u_th_C(C)

    R = Rf_uth(u,th)
    R = del_minz(R)
    return R


#Cuaternion from Rotation matrix
def C_R(R):
    
    w = 0.5*np.sqrt(1 + R[0,0] + R[1,1] + R[2,2])
    ex = (1/(4*w))*(R[2,1]-R[1,2])
    ey = (1/(4*w))*(R[0,2]-R[2,0])
    ez = (1/(4*w))*(R[1,0]-R[0,1])
    
    C = cuaternion(w,ex,ey,ez).del_minz_c()
    return C



#Vector rotations using Parameterizations
#Vector rotation given u,th
def rv_uth(v,u,th,au='r'):
    
    if au == 'd':
        th = np.deg2rad(th)

    u = unit_v(u)
    
    v_rot = v*cos(th) + np.cross(u,v.T).T*sin(th) + u.T*(np.dot(u,v))*(1-cos(th))

    return v_rot

#Vector rotation given C
def rv_C(v,C):

    w_v = 0
    ex_v = v[0,0]
    ey_v = v[1,0]
    ez_v = v[2,0]
    c_v = cuaternion(w_v,ex_v,ey_v,ez_v)
    
    c_v_rot = (C*c_v)*C.inv()

    v_rot = np.array([[c_v_rot.ex],[c_v_rot.ey],[c_v_rot.ez]])

    return v_rot 

#T given t,R
def T_tR(t,R):
    T = np.vstack((np.hstack((R,t)),np.array([[0, 0, 0, 1]])))
    return T

#R from T
def R_T(T):
    R = T[:3,:3]
    return R


def rotdh(d,th,a,alf):
    T=sp.Matrix([[cos(th),-cos(alf)*sin(th),sin(alf)*sin(th),a*cos(th)],
                  [sin(th),cos(alf)*cos(th),-sin(alf)*cos(th),a*sin(th)],
                  [0, sin(alf), cos(alf),d],[0, 0, 0, 1]])
    return T

def Jacobian(v_str, f_list):
    vars = sp.symbols(v_str)
    f = sp.sympify(f_list)
    J = sp.zeros(len(f),len(vars))
    for i, fi in enumerate(f):
        for j, s in enumerate(vars):
            J[i,j] = sp.diff(fi, s)
    return J

In [ ]:
def lista_de_funciones():
    return print("""Rotaciones canonicas \n
rotx,roty,rotz= Rotaciones canonicas alrededor del eje x, y y z
srotx, sroty, srotz= Rotaciones canonicas alrededor de los ejes x, y y z en forma simbolica
printr(value,n)= print con redondeo de n digitos
comprobar_so3(R,1-2 3-4)= Comprueba si una función pertenece a so3, rotacion(1-2) y transformacion homogenea(3-4)
rodr(th, u)= A partir de un u y un th te da una matriz de rotación
del_minz(R)= Elimina los valores de -0 en las matrices
unit_v(v)= Te da el vector unitario de un vector v
R_eulerangzyz(R,s)= A partir de una matriz de rotacion y un signo te da sus angulos de euler zyz
R_rpy(R,s)= A partir de una matriz de rotacion y un signo te da la representacion de roll, pitch, yaw
Rf_uth(u,th,au='r')= A partir de u y th, te da una matriz de rotacion, r representa radians y d degrees
u_th_R(R,s)= A partir de una matriz de rotacion, te da sus u y th
T_tR(t,R)= A partir del vector t y R te da una transformacion homogenea
R_T(T)= A partir de una transformacion homogenea te da una matriz de rotacion
##Crear cuaterniones y metodos para cuaterniones##
c=cuaternion(w,ex,ey,ez) = Crea un cuaternion (w,ex,ey,ez)
c.__mul__(c1,c2)= Te devuelve el producto de cuaterniones
c.inv(c)= Te devuelve la inversa del cuaternion c (w,-ex,-ey,-ez)
c.eye(c)= Te devuelve el cuaternion identidad
c.to_nparray(c)= Te devuelve el cuaternion como un np.array()
cuat_uth(u,th,au='r')= A partir de un u y th te regresa un cuaternion, en radianes por defecto, si lo quieres en degrees au='d'
cuat_rotxyz(th,c,au='r')= Para representar angulos en forma de cuaterniones, en x, y o z
u_th_C(C): A partir de un cuaternion, te da u y th
Rf_C(C)= Para pasar de cuaternion a matriz de rotacion
C_R(R)= A partir de una matriz de rotacion te da un cuaternion
rv_uth(v,u,th,au='r')= Si quieres rotar un vector con un eje y un angulo
rv_C(v,C)= Para rotar un vector v un cuaternion
rotdh(d,th,a,alf)=Obtienes la T de DH a partir de sus parametros""")
    

